## M&Ms


In [ ]:
### задача 4

set.seed(42)  # установим стартовое значение для генератора случайных чисел
x <- runif(200) # сгенерим 200 равномерных на [0;1]
x[1:5]

# функция ксюша возводит числа в квадрат
ksusha <- function(x) {
  return(x^2)
}

ksusha(7)

# логарифмическая функция правдоподобия для задачи 4 со знаком минус
log_lik <- function(a,dat) {
  n <- length(dat) # количество элементов в векторе dat
  res <- n*log(a) + (a-1)*sum(log(dat))
  return(-res)
}

# считаем для пробы значения нашей функции в некоторых точках
# числа 2 и 5 взяты от фонаря
log_lik(2,x)
log_lik(5,x)

# нелинейная оптимизация
?nlm

opt <- nlm(log_lik,p=5,hessian=TRUE,dat=x)
# p=5 - стартовое значение параметра, от которого R начинает искать экстремум.

a.hat <- opt$estimate # точка минимума функции
a.hat # оценка методом ML

var.hat <- 1/opt$hessian # матрица Гессе в точке минимума функции
var.hat # оценка дисперсии ML оценки


In [ ]:
########## 5

# генерим искусственные данные
x <- rnorm(200,mean=4,sd=3)

# функция правдоподобия зависит от двух параметров и вектора данных
# param - вектор из двух параметров
# data - вектор данных
log_lik <- function(param,data) {
  mu <- param[1]
  s2 <- param[2]
  n <- length(data)
  result <- -n/2*log(2*pi)-n/2*log(s2)-sum((data-mu)^2)/(2*s2)
  return(-result)
}

log_lik(c(3,5),x)

# результаты оптимизации
optimum <- nlm(log_lik,p=c(0,1),hessian=TRUE,data=x)
optimum

# вектор оценок неизвестных параметров
theta.hat <- optimum$estimate

# матрица Гессе в точке оптимума
H <- optimum$hessian
print(H,digits=2)

# оценка ковариационной матрицы - это обратная матрица к матрице Гессе
var.theta <- solve(H) 

# стандартные отклонения - это корни из диагональных элементов
st.devs <- sqrt(diag(var.theta))
st.devs
z.cr

ci.left <- theta.hat - z.cr*st.devs
ci.right <- theta.hat + z.cr*st.devs

ci.left
ci.right